In [2]:
import os
from src.solver import DetSolver
from src.core import YAMLConfig

config_path = 'configs/rtdetr/rtdetr_r18vd_6x_coco.yml'
checkpoint_url = "https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetr_r18vd_dec3_6x_coco_from_paddle.pth"

import torch
torch.hub.load_state_dict_from_url(checkpoint_url, "../models");

name = os.path.basename(checkpoint_url)
checkpoint_path = f"../models/{name}"

In [3]:
cfg = YAMLConfig(
    config_path,
    resume=checkpoint_path,
    # use_amp=args.amp,
    # tuning=args.tuning
)

solver = DetSolver(cfg)

In [4]:
solver.setup()
solver.resume(solver.cfg.resume)

Load PResNet18 state_dict
Loading ema.state_dict


In [5]:
from src.data.transforms import Resize, ToImageTensor, ConvertDtype, Compose
from PIL import Image, ImageOps

t = Compose(ops=[
    Resize([640, 640]),
    ToImageTensor(),
    ConvertDtype()
])

In [6]:
path = "image_02.jpg"

img = Image.open(path).convert("RGB")
try:
    img = ImageOps.exif_transpose(img)
except:
    print("No EXIF data")
    pass

w, h = img.size
orig_target_sizes = torch.as_tensor([int(w), int(h)]).unsqueeze(0)

In [12]:
device = "cuda:0"
model = solver.ema.module if solver.ema else solver.model
model.eval()
postprocessors = solver.postprocessor
# dataloader = None
with torch.no_grad():
    samples = t(img)[None]
    samples = samples.to(device)
    orig_target_sizes = orig_target_sizes.to(device)
    outputs = model(samples)
    results = postprocessors(outputs, orig_target_sizes)

In [8]:
postprocessors.num_top_queries = 5

In [10]:
import supervisely as sly

In [24]:
img = sly.image.read(path)
for box in results[0]['boxes'].cpu().numpy():
    x, y, x2, y2 = box
    sly.Rectangle(top=y, left=x, bottom=y2, right=x2).draw_contour(img, color=[255, 0, 0], thickness=3)

In [27]:
sly.image.write("out.jpg", img)

True